## Basic analysis of MTurk Experiments

In [28]:
# import
import json
import numpy as np
import pandas as pd
from datetime import datetime
from util import filters
from util.filters import FORMAT, LIKERT_CONVERTER, CONDITIONS

from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import seaborn as sns
DATEFORMAT = "%m/%d/%Y, %I:%M:%S %p"

In [29]:
file = "pilot_09_10_final.json"
with open("./prolific_logs/" + file) as f:
    logs = json.load(f)
logs = [logs["logs"][l] for l in logs["logs"]]

In [30]:
df = pd.read_csv("scored_data_v2.csv")

In [31]:
# helper function
def get_avg_std_err(vals):
    mean = np.mean(vals)
    std_dev = np.std(vals, ddof=1)  # Using ddof=1 for sample standard deviation
    
    # Calculate the standard error using the formula: standard deviation / sqrt(sample size)
    standard_error = std_dev / np.sqrt(len(vals))
    return mean, standard_error

#### Initial filters

In [32]:
# only consider completed sessions
print("Before filtering")
print("all logs:", len(logs))
for c in CONDITIONS:
    print(c, len([l for l in logs if l["condition"] == c]))


logs = [l for l in logs if l["completedSurvey"] == True]

print("\nAfter filtering")
print("completed logs:", len(logs))
for c in CONDITIONS:
    print(c, len([l for l in logs if l["condition"] == c]))

Before filtering
all logs: 166
reading 41
teacher-qa-bot 42
llm-qa-bot 41
llm-chatbot 42

After filtering
completed logs: 100
reading 19
teacher-qa-bot 29
llm-qa-bot 21
llm-chatbot 31


In [33]:
def get_blank_answer(log, q_id):
    first, second = "N/A", "N/A"
    if "answer" in log["knowledgeAnswers"][q_id]["answer"][0]:
        first = log["knowledgeAnswers"][q_id]["answer"][0]["answer"]
    if "answer" in log["knowledgeAnswers"][q_id]["answer"][1]:
        second = log["knowledgeAnswers"][q_id]["answer"][1]["answer"]
    return str((first, second)).replace("\t", "").replace("\n", " ")

def essay_answer(log, q_id):
    return log["knowledgeAnswers"][q_id]["answer"].replace("\t", "").replace("\n", " ")


csv_string = ""

header = "uuid\tcondition"
for i in range(6):
    idx = str(i + 1)
    header += "\t" + "Answer " + idx + "\tMeng " + idx + "\tRobin " + idx
csv_string += header + "\n"

for l in logs:
    c = l["condition"]
    line = ""
    csv_string += l["uuid"] + "\t" + c
    for q_id in ["q1", "q2", "q3"]:
        csv_string += "\t" + get_blank_answer(l, q_id) + "\t\t"
    for q_id in ["q4", "q5", "q6"]:
        csv_string += "\t" + essay_answer(l, q_id) + "\t\t"
    csv_string += line + "\n"

print(csv_string)
with open('dee_question_data.tsv', 'w') as file:
    file.write(csv_string)

n logs before filter: 100
reading 19
teacher-qa-bot 29
llm-qa-bot 21
llm-chatbot 31

n logs after filter: 54
reading 11
teacher-qa-bot 14
llm-qa-bot 10
llm-chatbot 19


In [ ]:
# NOTE: Filter sequence  | ADJUST AS DESIRED
print("n logs before filter:", len(logs))
for c in CONDITIONS:
    print(c, len([l for l in logs if l["condition"] == c]))

logs = [l for l in logs if filters.got_subject_right(l)]
logs = [l for l in logs if filters.got_subject_right(l)]
logs = [l for l in logs if filters.got_valid_degree(l)]
logs = [l for l in logs if filters.did_not_cheat(l)]
logs = [l for l in logs if filters.no_tab_switch_during_exam(l)]

print("\nn logs after filter:", len(logs))
for c in CONDITIONS:
    print(c, len([l for l in logs if l["condition"] == c]))

### Analyse deep responses

In [34]:
# WRITE A FUNCION THAT FOR EACH CONDTIONS PRINTS ALL THE ANSWERS
def get_condition(logs, c):
    return [l for l in logs if l["condition"] == c]

def print_blank_answer(logs, q_id):
    for l in logs:
        first, second = "N/A", "N/A"
        if "answer" in l["knowledgeAnswers"][q_id]["answer"][0]:
            first = l["knowledgeAnswers"][q_id]["answer"][0]["answer"]
        if "answer" in l["knowledgeAnswers"][q_id]["answer"][1]:
            second = l["knowledgeAnswers"][q_id]["answer"][1]["answer"]
        print(l["uuid"][:8], first, second)

def free_answers(logs, q_id):
    for l in logs:
        print(l["uuid"][:8], l["knowledgeAnswers"][q_id]["answer"])


In [35]:
# evaluate blank questions
for c in CONDITIONS:
    print(c)
    print("=================================")
    group = get_condition(logs, c)

    for q_id in ["q1", "q2", "q3"]:
        print(c, q_id)
        print_blank_answer(group, q_id)
        print("")
    print("")
    print("")
    print("=================================")

reading
reading q1
1d3c5ef8 MRNA protein which is used by the liver mainly 
1de97d07 mRNA protein synthesis
20e2e41a proteins mRNA
4a9770ff I'm not sure DNA
98f259f1 mRNA amino acids
bb00a8a0 mRNA molecules
cbcb5e82 mRNA amino acids
d611a8d0 mRNA amino acids
d79f7e95 mRNA whatever amino acid chain the nucleus ordered
dc335686 Phosphates Phenomes
f56852a3 rna mrma

reading q2
1d3c5ef8 Glucose ATP
1de97d07 Hydrogen protein
20e2e41a carbon hydrogen
4a9770ff Carbon Dioxide Not sure
98f259f1 glucose ATP
bb00a8a0 Carbon carbon dioxide
cbcb5e82 ATP lactic acid
d611a8d0 Glucose ATP
d79f7e95 H202 bruh what is this notation
dc335686 Hydrogen Carbon
f56852a3 carbon hydrogen

reading q3
1d3c5ef8 peroxisomes oxidation
1de97d07 the liver chemical
20e2e41a cells respiration
4a9770ff Cells Not sure
98f259f1 peoxisomes  oxidation
bb00a8a0 mitochondria N/A
cbcb5e82 peroxisomes chemical
d611a8d0 peroxisomes detoxification
d79f7e95 specific name specific name
dc335686 Ejaculation Photosynthesis
f56852a3 l

In [36]:
# evaluate free-form questions
for c in CONDITIONS:
    print(c)
    print("=================================")
    group = get_condition(logs, c)

    for q_id in ["q4", "q5", "q6"]:
        print(c, q_id)
        free_answers(group, q_id)
        print("")
    print("")
    print("")
    print("=================================")

reading
reading q4
1d3c5ef8 A cell has the capacity to alter its shape as well as its volume in response to need for altered force production. 
1de97d07 The number of cells and the organelles within depend on what the cells are expected to do--in this case make it possible for the organism to move.
20e2e41a Form follows function means that the format of cells is built so that they can function properly and help to power the human body through all phases of it's existence.
4a9770ff An example was given about a building that has all these components inside like an elevator and furniture. Cells and inside structure follow a similar pattern.
98f259f1 Form follows functions means that things, in this case, cells are built in a way that make sense for what they do, or how they function. It applies to the structure and function of muscles cells in that muscles cells contain a ot of mitochondria, which generate ATP (adenosine triphosphate) providing the energy needed to keep your body in motio

### ASSEMBLE RESULTS FOR MANUAL CODING

In [37]:
def get_blank_answer(log, q_id):
    first, second = "N/A", "N/A"
    if "answer" in log["knowledgeAnswers"][q_id]["answer"][0]:
        first = log["knowledgeAnswers"][q_id]["answer"][0]["answer"]
    if "answer" in log["knowledgeAnswers"][q_id]["answer"][1]:
        second = log["knowledgeAnswers"][q_id]["answer"][1]["answer"]
    return str((first, second)).replace("\t", "").replace("\n", " ")

def essay_answer(log, q_id):
    return log["knowledgeAnswers"][q_id]["answer"].replace("\t", "").replace("\n", " ")

def get_condition(logs, c):
    return [l for l in logs if l["condition"] == c]

In [38]:
csv_string = ""

header = "uuid\tcondition"
for i in range(6):
    idx = str(i + 1)
    header += "\t" + "Answer " + idx + "\tMeng " + idx + "\tRobin " + idx
csv_string += header + "\n"

for c in CONDITIONS:
    group = get_condition(logs, c)
    for l in group:
        if l["uuid"][:8] in df["uuid"].values:
            print("hit")
            continue
        line = ""
        csv_string += l["uuid"][:8] + "\t" + c
        for q_id in ["q1", "q2", "q3"]:
            csv_string += "\t" + get_blank_answer(l, q_id) + "\t\t"
        for q_id in ["q4", "q5", "q6"]:
            csv_string += "\t" + essay_answer(l, q_id) + "\t\t"
        csv_string += line + "\n"

print(csv_string)
with open('data.tsv', 'w') as file:
    file.write(csv_string)

hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
uuid	condition	Answer 1	Meng 1	Robin 1	Answer 2	Meng 2	Robin 2	Answer 3	Meng 3	Robin 3	Answer 4	Meng 4	Robin 4	Answer 5	Meng 5	Robin 5	Answer 6	Meng 6	Robin 6
1d3c5ef8	reading	('MRNA', 'protein which is used by the liver mainly ')			('Glucose', 'ATP')			('peroxisomes', 'oxidation')			A cell has the capacity to alter its shape as well as its volume in response to need for altered force production. 			The pancreas and its cells are important for digesting food and managing your use of sugar for energy atfer digestion. If it gets out of wack you can get diabetes a very serious disease that can lead to loss of life			the catalase they contain.		
4a9770ff	reading	("I'm not sure", 'DNA')			('Carbon Dioxide', 'Not sure')			('Cells', 'Not sure')			An example was given about a building that has all these components inside like an elevator and furniture. Cel

In [39]:
# tab switches
def exam_switches(log):
    switches = 0
    knowledge = datetime.strptime(log["timestamps"]["enterKnowledge"], DATEFORMAT)
    survey = datetime.strptime(log["timestamps"]["enterSurvey"], DATEFORMAT)
    if not "tabSwitches" in log:
        return 0
    for switch in log["tabSwitches"]:
        if switch[0] == "exit":
            time = datetime.strptime(switch[1], DATEFORMAT)
            if (knowledge < time) and (time < survey):
                switches += 1
    return switches

filtered_logs = []
for c in ['llm-chatbot', 'llm-qa-bot', 'teacher-qa-bot', 'reading']:
    print(c)
    group = [l for l in logs if l["condition"] == c]
    for log in group:
        print(exam_switches(log))
        if exam_switches(log) == 0:
            filtered_logs.append(log)
    print("")
# logs = filtered_logs

llm-chatbot
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

llm-qa-bot
0
0
0
0
0
0
0
0
0
0

teacher-qa-bot
0
0
0
0
0
0
0
0
0
0
0
0
0
0

reading
0
0
0
0
0
0
0
0
0
0
0



In [40]:
# attention detected
def cursor_left(log):
    switches = 0
    knowledge = datetime.strptime(log["timestamps"]["enterKnowledge"], DATEFORMAT)
    survey = datetime.strptime(log["timestamps"]["enterSurvey"], DATEFORMAT)
    if "modalTimes" not in log:
        return 0
    for switch in log["modalTimes"]:
        if switch[0] == "exit":
            time = datetime.strptime(switch[1], DATEFORMAT)
            if (knowledge < time) and (time < survey):
                switches += 1
    return switches

filtered_logs = []
for c in ['llm-chatbot', 'llm-qa-bot', 'teacher-qa-bot', 'reading']:
    print(c)
    group = [l for l in logs if l["condition"] == c]
    for log in group:
        print(cursor_left(log))
        if cursor_left(log) == 0:
            filtered_logs.append(log)
    print("")
# logs = filtered_logs

llm-chatbot
1
0
7
0
0
5
0
0
0
0
0
0
1
2
0
0
0
0
0

llm-qa-bot
0
2
2
0
0
0
2
0
0
1

teacher-qa-bot
0
0
1
2
2
1
1
7
2
3
0
0
1
0

reading
6
1
2
1
1
2
0
0
1
1
1



In [41]:
# count paste events
def count_paste(log):
    paste_count = 0
    if "knowledgePaste" not in log:
        return 0
    for k in log["knowledgePaste"]:
        paste_count += len(log["knowledgePaste"][k])
        print(log["knowledgePaste"][k])
    return paste_count

for c in ['llm-chatbot', 'llm-qa-bot', 'teacher-qa-bot', 'reading']:
    print(c)
    group = [l for l in logs if l["condition"] == c]
    for log in group:
        print(count_paste(log))
    print("")
# logs = filtered_logs

llm-chatbot
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[{'text': 'by oxidation reactions that are confined in the peroxisomes. ', 'timestamp': '9/10/2023, 3:24:56 PM'}, {'text': 'by oxidation reactions that are confined in the peroxisomes. ', 'timestamp': '9/10/2023, 3:24:57 PM'}]
2
0

llm-qa-bot
0
0
0
0
0
0
0
0
0
0

teacher-qa-bot
0
0
0
0
0
0
[{'text': 'oxidation reactions', 'timestamp': '9/10/2023, 9:49:49 PM'}, {'text': 'oxidation reactions', 'timestamp': '9/10/2023, 9:49:51 PM'}, {'text': 'oxidation reactions', 'timestamp': '9/10/2023, 9:49:55 PM'}]
3
0
0
[{'text': 'The principle of “form follows function” suggests that the structure of a biological component is closely related to its specific role. In muscle cells, this principle is evident as their long, cylindrical shape with many contractile fibers, which enables efficient contraction and relaxation necessary for movement.', 'timestamp': '9/10/2023, 10:12:19 PM'}, {'text': 'The principle of “form follows function” suggests that the stru

In [42]:
knowledgePaste

NameError: name 'knowledgePaste' is not defined

In [ ]:
# print free response questions
for c in ['llm-chatbot', 'llm-qa-bot', 'teacher-qa-bot', 'reading']:
    print(c)
    group = [l for l in logs if l["condition"] == c]
    for i, e in enumerate(group):
        if "answer" in group[i]["surveyAnswers"]["q15"]:
            print(i, group[i]["surveyAnswers"]["q15"]["answer"])
    print("")
